# Harvest all your hacks

A Python 3 script which collects projects from a Dribdat instance. Just put in the URL of a server, and it will talk to the public API to collect the content.

No special dependencies required, though you might need to `pip install requests`

In [1]:
import csv, json, requests

URL_DRIBDAT = 'https://hack.opendata.ch'
OUTPUT_PROJECTS = 'data/hackopendata-projects.csv'
OUTPUT_EVENTS = 'data/hackopendata-events.csv'
MAX_PROJECTS = 100000

Get the complete list of events on the server defined above

In [2]:
events_data = requests.get(URL_DRIBDAT + '/api/events.json').json()
events = events_data['events']
len(events)

45

Collect project data from each event

In [3]:
count_total = 0
project_data = {}
for event in events:
    urlapi = URL_DRIBDAT + "/api/event/%d/projects.json" % event['id']
    proj_data = requests.get(urlapi).json()
    if 'projects' in proj_data:
        project_data[event['id']] = proj_data['projects']
        count_total = count_total + len(proj_data['projects'])
    else:
        project_data[event['id']] = []
        print('No data for event %d' % event['id'])
    if count_total > MAX_PROJECTS: break
count_total

676

Load in the metadata schema from the Data Package

In [10]:
projcols = eventcols = None
with open('datapackage.json') as f:
    for res in json.load(f)['resources']:
        if 'events' in res['name']:
            eventcols = [ f['name'] for f in res['schema']['fields'] ]
        if 'projects' in res['name']:
            projcols = [ f['name'] for f in res['schema']['fields'] ]
','.join(eventcols)

'community_url,ends_at,gallery_url,has_finished,has_started,hashtags,hostname,id,location,location_lat,location_lon,logo_url,name,starts_at,summary,webpage_url'

In [6]:
with open(OUTPUT_EVENTS, "w") as f:
    cw = csv.DictWriter(f, eventcols, delimiter=',')
    cw.writeheader()
    cw.writerows(events)
"Wrote %s" % OUTPUT_EVENTS

'Wrote data/events.csv'

In [4]:
all_projects = []
for pd in project_data:
    for proj in project_data[pd]:
        all_projects.append(proj)
len(all_projects)

676

In [11]:
with open(OUTPUT_PROJECTS, "w") as f:
    cw = csv.DictWriter(f, projcols, delimiter=',')
    cw.writeheader()
    cw.writerows(all_projects)
"Wrote %s" % OUTPUT_PROJECTS

'Wrote data/projects.csv'

![](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fvignette3.wikia.nocookie.net%2Fffxiv-new%2Fimages%2F8%2F84%2FAll-your-base-are-belong-to-us-o.gif%2Frevision%2Flatest%3Fcb%3D20150609194420&f=1&nofb=1&ipt=60df4e7eac07ffec69db0105e478dd729caa15d4b7a30bbbb05bded32256a168&ipo=images)